<H1> PyCity Schools Challenge

<p>Author: Alex Schanne
<p>This notebook will analyze school performances within PyCity. It will focus on math and reading performance using grades as the metric for evaluation. It will also consider school spending, size and type (i.e. charter vs district) as factors of performance.

In [1]:
#importing dependencies
import pandas as pd
import numpy as np

In [2]:
#Creating path to csv and read it into Pandas DataFrame
schools_path = "Resources/schools_complete.csv"
students_path = "Resources/students_complete.csv"

schools_df = pd.read_csv(schools_path)
students_df = pd.read_csv(students_path)

In [3]:
#merging the two datasets
school_complete = pd.merge(students_df, schools_df, how = "left", on = ["school_name", "school_name"])
school_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## School Summary

Now that we have set up the data properly and merged the two datasets, we will create a summary for the district. It will include: 
    <li> Total Number of Schools
    <li> Total Number of Students
    <li> Total Budget
    <li> Average Math Score
    <li> Average Reading Score
    <li> The percentage of students with a passing math score (70% or higher)
    <li> The percentage of students with a passing reading score (70% or higher)
    <li> The percentage of students passing math and reading (% Overall passing)
        
 <p> <b> Please note that this data may have errors in it. It was not cleaned for anomalies.

In [4]:
#Calculating the needed information for the school summary data and creating the dataframe
tot_schools = school_complete["school_name"].nunique()
tot_students = school_complete["student_name"].count()
tot_budget = schools_df["budget"].sum()
avg_math = school_complete["math_score"].mean()
avg_read = school_complete["reading_score"].mean()
math_pass = (len(school_complete[school_complete["math_score"] >= 70]))/tot_students * 100
read_pass = (len(school_complete[school_complete["reading_score"] >= 70]))/tot_students * 100
overall_pass = len(school_complete.loc[(school_complete["math_score"] >= 70) & (school_complete["reading_score"] >= 70)])/tot_students * 100

#Creating the DataFrame and formatting it for a cleaner picture of the data
school_sum =[{"Total Schools":tot_schools,
              "Total Students": '{:,}'.format(tot_students), 
              "Total Budget": '${:,.2f}'.format(tot_budget), 
              "Average Math Score": '{:,.2f}'.format(avg_math), 
              "Average Reading Score": '{:,.2f}'.format(avg_read), 
              "% Passing Math": '{:.2f}%'.format(math_pass), 
              "% Passing Reading": '{:.2f}%'.format(read_pass), 
              "% Passing Overall":'{:.2f}%'.format(overall_pass)}]
sum_df = pd.DataFrame(school_sum)
sum_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


Now that we have created the District wide summary. We will look at a summary of the district, broken down by school. Having this will make it easier to analyze school performance according to our key metrics. 

In [9]:
#Calculating and creating a dataframe grouped by each school
group_school = school_complete.set_index("school_name").groupby(["school_name"])

typeschool = schools_df.set_index(['school_name'])['type']
sch_stud = group_school['Student ID'].count()
sch_budg = schools_df.set_index('school_name')['budget']
budg_per_stud = sch_budg/sch_stud
sch_avgmath = group_school['math_score'].mean()
sch_avgread = group_school['reading_score'].mean()
sch_mathpass = school_complete[school_complete["math_score"] >= 70].groupby(['school_name'])['Student ID'].count()
per_mathpass = sch_mathpass/sch_stud * 100
sch_readpass = school_complete[school_complete["reading_score"] >= 70].groupby(['school_name'])['Student ID'].count()/sch_stud * 100
sch_overall = school_complete[(school_complete["math_score"] >= 70) & (school_complete["reading_score"] >= 70)].groupby(['school_name'])['Student ID'].count()/sch_stud * 100

#formatting and outputting the dataframe
schools = [{"School Type":typeschool,
            "Total Students":sch_stud,
            "Total School Budget":sch_budg,
            "Per Student Budget": budg_per_stud,
            "Average Math Score":sch_avgmath,
            "Average Reading Score":sch_avgread,
            "% Passing Math":per_mathpass,
            "% Passing Reading":sch_readpass,
            "Overall Passing Rate":sch_overall}]

schools_summary = pd.DataFrame(schools)

schools_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,school_name Huang High School District ...,school_name Bailey High School 4976 Cabr...,school_name Huang High School 1910635 F...,school_name Bailey High School 628.0 Cab...,school_name Bailey High School 77.048432...,school_name Bailey High School 81.033963...,school_name Bailey High School 66.680064...,school_name Bailey High School 81.933280...,school_name Bailey High School 54.642283...


## Top Performing Schools (By % Overall Passing)

These are the top five performing schools as determined by overall percent of passing students.

In [ ]:
#using the previously created schools_summary dataframe and sorting it by overall passing percentage

top_overallpass = schools_summary.sort_values("Overall Passing Rate", ascending = False)
top_overallpass.head(5)

## Bottom Performing Schools (By % Overall Passing)

These are the worst performing schools as determined by overall percentage of passing students.

In [ ]:
#using the previously defined schools dataframe and sorting it by descending value order

bottom_overallpass = schools_summary.sort_values("Overall Passing Rate", ascending = True)
bottom_overallpass.head(5)

## Math Scores by Grade

The following work will provide average math scores for each grade level (9th through 12th grade) as reported for the schools in the district.

In [ ]:
#math scores by grade level
nine_math = students_df.loc[students_df['grade']== "9th"].groupby(['school_name'])['math_score'].mean()
ten_math = students_df.loc[students_df['grade']== "10th"].groupby(['school_name'])['math_score'].mean()
eleven_math = students_df.loc[students_df['grade']== "11th"].groupby(['school_name'])['math_score'].mean()
twelve_math = students_df.loc[students_df['grade']== "12th"].groupby(['school_name'])['math_score'].mean()

math = pd.DataFrame({
    "9th Grade": nine_math,
    "10th Grade": ten_math,
    "11th Grade": eleven_math,
    "12th Grade": twelve_math
})

math = math[['9th Grade', '10th Grade', '11th Grade', '12th Grade']]
math.index.name = "School"
math

## Reading Score by Grade

The following work will provide average reading scores for each grade level (9th through 12th grade) as reported for the schools in the district.

In [ ]:
#reading scores by grade level
nine_read = students_df.loc[students_df['grade']== "9th"].groupby(['school_name'])['reading_score'].mean()
ten_read = students_df.loc[students_df['grade']== "10th"].groupby(['school_name'])['reading_score'].mean()
eleven_read = students_df.loc[students_df['grade']== "11th"].groupby(['school_name'])['reading_score'].mean()
twelve_read = students_df.loc[students_df['grade']== "12th"].groupby(['school_name'])['reading_score'].mean()

read = pd.DataFrame({
    "9th Grade": nine_read,
    "10th Grade": ten_read,
    "11th Grade": eleven_read,
    "12th Grade": twelve_read
})

read = read[['9th Grade', '10th Grade', '11th Grade', '12th Grade']]
read.index.name = "School"
read

## Scores by School Spending

The following will analyze school performance based on average spending by number of students.
In order to do so, the data will be binned into ranges of school spending.

In [ ]:
#Creating the bins in which the data will be held
bin = [0, 584, 629, 644, 675]
bin_names = ["<$584", "$585-$629", "$630-$644", "$645-$675"]
school_complete['budg_bin'] = pd.cut(school_complete['budget']/school_complete['size'], bin, labels = bin_names)
bin_spend = school_complete.groupby('budg_bin')


#Calculating the data to go in the new binned DataFrame
avgmath = bin_spend['math_score'].mean()
avgread = bin_spend['reading_score'].mean()
passmath = school_complete[school_complete['math_score'] >= 70].groupby(['budg_bin'])['Student ID'].count()/bin_spend['Student ID'].count() *100
passread = school_complete[school_complete['reading_score'] >= 70].groupby(['budg_bin'])['Student ID'].count()/bin_spend['Student ID'].count() * 100
overpass = school_complete[(school_complete['math_score'] >= 70) & (school_complete['reading_score'] >= 70)].groupby(['budg_bin'])['Student ID'].count()/bin_spend['Student ID'].count() * 100

#Creating teh new binned DataFrame
bin_by_spend = pd.DataFrame({"Average Math Score": avgmath,
                            "Average Reading Score": avgread,
                            "% Passing Math": passmath,
                            "% Passing Reading": passread,
                            "Overall % Passing": overpass})

bin_by_spend

## Scores by School Size

The following will analyze school performances based on school sizes. Again, this analysis will require binning the data into groups. In this instance, we will create three bins to represent small (<1000), medium (1000-2000), or large (>2000) schools. 

In [ ]:
#Creating the bins in which the data will be held
sizebins = [0, 999, 1999, 9999999999]
size_names = ["Small (<1000)", "Medium (1000-2000)" , "Large (>2000)"]
school_complete['size_bins'] = pd.cut(school_complete['size'], sizebins, labels = size_names)
sizes = school_complete.groupby('size_bins')
 
avgmath = sizes['math_score'].mean()
avgread = sizes['reading_score'].mean()
passmath = school_complete[school_complete['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/sizes['Student ID'].count() * 100
passread = school_complete[school_complete['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/sizes['Student ID'].count() * 100
overpass = school_complete[(school_complete['reading_score'] >= 70) & (school_complete['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/sizes['Student ID'].count() * 100
            
bin_by_size = pd.DataFrame({"Average Math Score": avgmath,
    "Average Reading Score": avgread,
    '% Passing Math': passmath,
    '% Passing Reading': passread,
    "Overall Passing Rate": overpass
            
})          
bin_by_size

## Scores by School Type

The following will analyze school performances based on school type (District vs. Charter)

In [ ]:
#Grouping the school district by typ of school and analyzing performance
types = school_complete.groupby('type')
 
avgmath = types['math_score'].mean()
avgread = types['reading_score'].mean()
avgbudg = types['budget'].mean()
passmath = school_complete[school_complete['math_score'] >= 70].groupby('type')['Student ID'].count()/types['Student ID'].count() * 100
passread = school_complete[school_complete['reading_score'] >= 70].groupby('type')['Student ID'].count()/types['Student ID'].count() * 100
overpass = school_complete[(school_complete['reading_score'] >= 70) & (school_complete['math_score'] >= 70)].groupby('type')['Student ID'].count()/types['Student ID'].count() * 100
            
grouptype = pd.DataFrame({"Average Budget": '${:,.2f}'.format(avgbudg),
    "Average Math Score": avgmath,
    "Average Reading Score": avgread,
    '% Passing Math': passmath,
    '% Passing Reading': passread,
    "Overall Passing Rate": overpass
            
})          
grouptype


# Written Observations